In [1]:
# https://trueskill.org/


In [2]:
from trueskill import Rating, quality_1vs1, rate_1vs1

BETA = 8.333  # 
alice, bob = Rating(25), Rating(25 + BETA)  # assign Alice and Bob's ratings
alice, bob

(trueskill.Rating(mu=25.000, sigma=8.333),
 trueskill.Rating(mu=33.333, sigma=8.333))

In [3]:
def win_probability(greater, lesser):
    # # https://github.com/sublee/trueskill/issues/1
    exp = (greater.mu - lesser.mu) / BETA
    n = 4. ** exp
    return n / (n + 1)

In [4]:
win_probability(bob, alice), win_probability(alice, bob)

(0.8, 0.20000000000000004)

In [5]:
alice, bob = rate_1vs1(alice, bob)  # param 1 is the winner of the match

In [6]:
alice, bob

(trueskill.Rating(mu=31.756, sigma=6.952),
 trueskill.Rating(mu=26.577, sigma=6.952))

# example
how quickly can trueskill ratings converge?

In [44]:
import numpy as np
from trueskill import Rating, quality_1vs1, rate_1vs1
from statistics import NormalDist
import matplotlib.pyplot as plt
%matplotlib inline

BETA = 8.333

def win_probability(greater, lesser):
    # # https://github.com/sublee/trueskill/issues/1
    exp = (greater.mu - lesser.mu) / BETA
    n = 4. ** exp
    return n / (n + 1)

In [41]:
class Player(object):
    def __init__(self, name, mu):
        self.name = name
        self.real = Rating(mu)
        self.fake = Rating(50)  # everyone starts off with the mean rating
        self.count = 0

In [69]:
# create a pool of players

n_players = 20

players = {}
for i in range(n_players):
    m = np.random.uniform(0, 100)
    players[i] = Player(name=i, mu=m)

# https://stackoverflow.com/questions/32551610/overlapping-probability-of-two-normal-distribution-with-scipy
average_overlap = []
for k, v in players.items():
    print(v.name, v.real, v.fake)
    average_overlap.append(NormalDist(mu=v.real.mu, sigma=v.real.sigma).overlap(NormalDist(mu=v.fake.mu, sigma=v.fake.sigma)))

similarity = np.mean(average_overlap)
print("similarity", similarity)

0 trueskill.Rating(mu=57.737, sigma=8.333) trueskill.Rating(mu=50.000, sigma=8.333)
1 trueskill.Rating(mu=40.001, sigma=8.333) trueskill.Rating(mu=50.000, sigma=8.333)
2 trueskill.Rating(mu=37.633, sigma=8.333) trueskill.Rating(mu=50.000, sigma=8.333)
3 trueskill.Rating(mu=23.917, sigma=8.333) trueskill.Rating(mu=50.000, sigma=8.333)
4 trueskill.Rating(mu=88.493, sigma=8.333) trueskill.Rating(mu=50.000, sigma=8.333)
5 trueskill.Rating(mu=86.894, sigma=8.333) trueskill.Rating(mu=50.000, sigma=8.333)
6 trueskill.Rating(mu=87.334, sigma=8.333) trueskill.Rating(mu=50.000, sigma=8.333)
7 trueskill.Rating(mu=0.166, sigma=8.333) trueskill.Rating(mu=50.000, sigma=8.333)
8 trueskill.Rating(mu=30.553, sigma=8.333) trueskill.Rating(mu=50.000, sigma=8.333)
9 trueskill.Rating(mu=39.850, sigma=8.333) trueskill.Rating(mu=50.000, sigma=8.333)
10 trueskill.Rating(mu=92.114, sigma=8.333) trueskill.Rating(mu=50.000, sigma=8.333)
11 trueskill.Rating(mu=0.377, sigma=8.333) trueskill.Rating(mu=50.000, sigma

In [70]:
n_iterations = 30


x = np.linspace(-20, 120, 100)
from scipy import stats

for iteration in range(n_iterations):
    a, b = np.random.choice(range(n_players), 2)
    p1 = players[a]
    p2 = players[b]
    
    p1.count += 1
    p2.count += 1

    prob_p1win = win_probability(p1.real, p2.real)

    # p1 wins
    if np.random.uniform() < prob_p1win:
        p1.fake, p2.fake = rate_1vs1(p1.real, p2.real)
    else:
        p2.fake, p1.fake = rate_1vs1(p2.real, p1.real)


In [71]:
average_overlap = []
for k, v in players.items():
    print(v.name, v.real, v.fake)
    average_overlap.append(NormalDist(mu=v.real.mu, sigma=v.real.sigma).overlap(NormalDist(mu=v.fake.mu, sigma=v.fake.sigma)))

similarity = np.mean(average_overlap)
print("similarity", similarity)

0 trueskill.Rating(mu=57.737, sigma=8.333) trueskill.Rating(mu=58.506, sigma=7.931)
1 trueskill.Rating(mu=40.001, sigma=8.333) trueskill.Rating(mu=40.029, sigma=8.308)
2 trueskill.Rating(mu=37.633, sigma=8.333) trueskill.Rating(mu=37.586, sigma=8.292)
3 trueskill.Rating(mu=23.917, sigma=8.333) trueskill.Rating(mu=24.394, sigma=8.052)
4 trueskill.Rating(mu=88.493, sigma=8.333) trueskill.Rating(mu=88.633, sigma=8.229)
5 trueskill.Rating(mu=86.894, sigma=8.333) trueskill.Rating(mu=83.759, sigma=7.349)
6 trueskill.Rating(mu=87.334, sigma=8.333) trueskill.Rating(mu=87.334, sigma=8.334)
7 trueskill.Rating(mu=0.166, sigma=8.333) trueskill.Rating(mu=-0.004, sigma=8.211)
8 trueskill.Rating(mu=30.553, sigma=8.333) trueskill.Rating(mu=30.722, sigma=8.211)
9 trueskill.Rating(mu=39.850, sigma=8.333) trueskill.Rating(mu=40.003, sigma=8.221)
10 trueskill.Rating(mu=92.114, sigma=8.333) trueskill.Rating(mu=95.249, sigma=7.349)
11 trueskill.Rating(mu=0.377, sigma=8.333) trueskill.Rating(mu=0.350, sigma=